In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from qvix import get_qvix, update_qvix, get_hist_vol
from option_backtest import OptionBacktest

In [ ]:
qvix = update_qvix()

In [ ]:
qvix_df = get_qvix()
# qvix_df.describe()

In [ ]:
fig = px.box(qvix_df, y='high', points='all')
fig.show()

In [ ]:
hist_vol = get_hist_vol()
columns = [name for name in hist_vol.columns if name.startswith('HV')]
quantile_df = hist_vol[columns].quantile(np.arange(.1, 1.1, .1))
quantile_df.index = list(range(10, 110, 10))

In [ ]:
quantile_df

In [ ]:
fig = go.Figure()

for index, name in quantile_df.iterrows():
    fig.add_trace(go.Scatter(x=columns, y=quantile_df.loc[index].values, mode='lines+markers', name=str(index)))

fig.show()

In [ ]:
params = [
    {'hedge_range': 0.1, 'hedge_value': 0.05},
    {'hedge_range': 0.1, 'hedge_value': 0.1},
    {'hedge_range': 0.075, 'hedge_value': 0.0375},
    {'hedge_range': 0.075, 'hedge_value': 0.075},
    {'hedge_range': 0.05, 'hedge_value': 0.025},
    {'hedge_range': 0.05, 'hedge_value': 0.05},
    {'hedge_range': 0.03, 'hedge_value': 0.015},
    {'hedge_range': 0.03, 'hedge_value': 0.03},
]

starts = ['2005-02-23', '2010-01-01', '2005-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01']

for start in starts:
    for param in params:
        backtest = OptionBacktest(start=start)
        backtest.set_parameters(param)
        backtest.load_data()
        backtest.run_backtest()
        backtest.stats_result()
    print('=' * 50)

